In [11]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'
# %load_ext line_profiler # Example: %lprun -T lprof0 -f celeri.celeri.get_okada_displacements u_east, u_north, u_up = celeri.get_okada_displacements(segment.lon1.values[segment_idx], segment.lat1[segment_idx], segment.lon2[segment_idx], segment.lat2[segment_idx], segment.locking_depth[segment_idx], segment.burial_depth[segment_idx], segment.dip[segment_idx], command.material_lambda, command.material_mu, 1, 0, 0, station.lon, station.lat)

In [12]:
import addict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [13]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

In [ ]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this work correctly

# Build all linear operators
operators.block_rotation = celeri.get_block_rotation_operator(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_operator(station)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)
# celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
operators.block_strain_rate, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment)
operators.mogi_station = celeri.get_mogi_operator(mogi, station, command)

# Get additiona matrix shape information for assembly
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct

In [6]:
# Starting work on TDEs
import matplotlib.pyplot as plt
import numpy as np
import cutde

xs = np.linspace(-2, 2, 200)
ys = np.linspace(-2, 2, 200)
obsx, obsy = np.meshgrid(xs, ys)
pts = np.array([obsx, obsy, 0 * obsy]).reshape((3, -1)).T.copy()

fault_pts = np.array([[-1, 0, 0], [1, 0, 0], [1, 0, -1], [-1, 0, -1]])
fault_tris = np.array([[0, 1, 2], [0, 2, 3]], dtype=np.int64)

%time disp_mat = cutde.fullspace.disp_matrix(obs_pts=pts, tris=fault_pts[fault_tris], nu=0.25)

slip = np.array([[1, 0, 0], [1, 0, 0]])
disp = disp_mat.reshape((-1, 6)).dot(slip.flatten())
disp_grid = disp.reshape((*obsx.shape, 3))

plt.figure()
cntf = plt.contourf(obsx, obsy, disp_grid[:, :, 0], levels=21)
plt.contour(obsx, obsy, disp_grid[:, :, 0], colors="k", linestyles="-", linewidths=0.5,levels=21)
plt.colorbar(cntf)
plt.title("u_x")
plt.tight_layout()
# plt.savefig("docs/example.png", bbox_inches="tight")

CPU times: user 248 ms, sys: 5.98 ms, total: 254 ms
Wall time: 28.4 ms


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:

segment_locking_depth *= celeri.KM2M
segment_burial_depth *= celeri.KM2M

# Project coordinates to flat space using a local oblique Mercator projection
projection = celeri.get_segment_oblique_projection(segment_lon1, segment_lat1, segment_lon2, segment_lat2)
station_x, station_y = projection(station_lon, station_lat)
segment_x1, segment_y1 = projection(segment_lon1, segment_lat1)
segment_x2, segment_y2 = projection(segment_lon2, segment_lat2)

# Translate stations and segment so that segment mid-point is at the origin
segment_x_mid = (segment_x1 + segment_x2) / 2.0
segment_y_mid = (segment_y1 + segment_y2) / 2.0
station_x -= segment_x_mid
station_y -= segment_y_mid
segment_x1 -= segment_x_mid
segment_x2 -= segment_x_mid
segment_y1 -= segment_y_mid
segment_y2 -= segment_y_mid

# Unrotate coordinates to eliminate strike, segment will lie along y = 0
rotation_matrix = np.array(
    [
        [np.cos(segment_strike), -np.sin(segment_strike)],
        [np.sin(segment_strike), np.cos(segment_strike)],
    ]
)
station_x_rotated, station_y_rotated = np.hsplit(np.einsum("ij,kj->ik", np.dstack((station_x, station_y))[0], rotation_matrix.T), 2)

# Elastic displacements from Okada 1992
u_x = np.zeros_like(station_x)
u_y = np.zeros_like(station_x)
u_up = np.zeros_like(station_x)
for i in range(len(station_x)):
    _, u, _ = okada_wrapper.dc3dwrapper(
        alpha,  # (lambda + mu) / (lambda + 2 * mu)
        [
            station_x_rotated[i],
            station_y_rotated[i],
            0,
        ],  # (meters) observation point
        -segment_locking_depth,  # (meters) depth of the fault origin
        segment_dip,  # (degrees) the dip-angle of the rectangular dislocation surface
        [
            -segment_length / 2,
            segment_length / 2,
        ],  # (meters) the along-strike range of the surface (al1,al2 in the original)
        [
            0,
            segment_up_dip_width,
        ],  # (meters) along-dip range of the surface (aw1, aw2 in the original)
        [strike_slip, dip_slip, tensile_slip],
    )  # (meters) strike-slip, dip-slip, tensile-slip
    u_x[i] = u[0]
    u_y[i] = u[1]
    u_up[i] = u[2]

# Un-rotate displacement to account for projected fault strike
u_east, u_north = np.hsplit(np.einsum("ij,kj->ik", np.dstack((u_x, u_y))[0], rotation_matrix), 2)


(40000, 3)

In [70]:
import pyproj
def get_transverse_projection(lon0, lat0):
    """
    Use pyproj oblique mercator: https://proj.org/operations/projections/tmerc.html
    """
    if lon0 > 180.0:
        lon0 = lon0 - 360
    projection_string = (
        "+proj=tmerc "
        + "+lon_0="
        + str(lon0)
        + " "
        + "+lat_0="
        + str(lat0)
        + " "
        + "+ellps=WGS84"
    )
    projection = pyproj.Proj(pyproj.CRS.from_proj4(projection_string))
    return projection


tri_idx = 0
plt.figure()
plt.plot(station.lon, station.lat, "r.")
plt.plot(meshes[0].lon1[tri_idx], meshes[0].lat1[tri_idx], "+k")
plt.plot(meshes[0].lon2[tri_idx], meshes[0].lat2[tri_idx], "+k")
plt.plot(meshes[0].lon3[tri_idx], meshes[0].lat3[tri_idx], "+k")
plt.show()

# Try a projected version
tri_centroid_lon = meshes[0].centroids[tri_idx, 0]
tri_centroid_lat = meshes[0].centroids[tri_idx, 1]
projection = get_transverse_projection(tri_centroid_lon, tri_centroid_lat)
station_x, station_y = projection(station.lon, station.lat)
tri_x1, tri_y1 = projection(meshes[0].lon1[tri_idx], meshes[0].lat1[tri_idx])
tri_x2, tri_y2 = projection(meshes[0].lon2[tri_idx], meshes[0].lat2[tri_idx])
tri_x3, tri_y3 = projection(meshes[0].lon3[tri_idx], meshes[0].lat3[tri_idx])
tri_z1 = celeri.KM2M * meshes[0].dep1[tri_idx]
tri_z2 = celeri.KM2M * meshes[0].dep2[tri_idx]
tri_z3 = celeri.KM2M * meshes[0].dep3[tri_idx]

plt.figure()
plt.plot(station_x, station_y, "r.")
plt.plot(tri_x1, tri_y1, "+k")
plt.plot(tri_x2, tri_y2, "+k")
plt.plot(tri_x3, tri_y3, "+k")
plt.show()

# Package coordinates for cutde call
obs_coords = np.vstack((station_x, station_y, np.zeros_like(station_x))).T
# tri_coords = np.array([[-1, 0, 0], [1, 0, 0], [1, 0, -1]])
tri_coords = np.array([[tri_x1, tri_y1, tri_z1], [tri_x2, tri_y2, tri_z2], [tri_x3, tri_y3, tri_z3]])
fault_tris = np.array([[0, 1, 2]], dtype=np.int64)
# %time disp_mat = cutde.fullspace.disp_matrix(obs_pts=pts, tris=fault_pts[fault_tris], nu=0.25)
%time disp_mat = cutde.fullspace.disp_matrix(obs_pts=obs_coords, tris=np.array([tri_coords]), nu=0.25)
# %time disp_mat = cutde.fullspace.disp_matrix(obs_pts=obs_coords, tris=fault_pts[fault_tris], nu=0.25)

slip = np.array([[1, 0, 0]])
disp = disp_mat.reshape((-1, 3)).dot(slip.flatten())
u_x = disp[0::3]
u_y = disp[1::3]
u_z = disp[2::3]

plt.figure()
plt.plot(station_x, station_y, "r.")
plt.plot(tri_x1, tri_y1, "+k")
plt.plot(tri_x2, tri_y2, "+k")
plt.plot(tri_x3, tri_y3, "+k")
plt.quiver(station_x, station_y, u_x, u_y, scale=1e7, scale_units="inches",)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

CPU times: user 6.81 ms, sys: 1.25 ms, total: 8.06 ms
Wall time: 2.12 ms


/Users/meade/opt/anaconda3/envs/celeri/lib/python3.9/site-packages/cutde/coordinators.py:77: UserWarning: The obs_pts input array has Fortran ordering. Converting to C ordering. This may be expensive.
  warnings.warn(


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …